In [1]:
!pip install -q datasets==3.2.0 bitsandbytes==0.45.1 accelerate==1.2.1 evaluate==0.4.3 \
    transformers==4.47.1 torch==2.5.1+cu124 numpy==1.26.4 peft==0.14.0 sacrebleu

In [2]:
import os
import torch
import numpy as np
import evaluate
from datasets import load_dataset
from transformers import (
    AutoModelForSeq2SeqLM,
    AutoTokenizer,
    TrainingArguments,
    Trainer,
    DataCollatorForSeq2Seq,
    BitsAndBytesConfig
)
from peft import (
    LoraConfig,
    get_peft_model,
    TaskType,
    prepare_model_for_kbit_training
)

In [3]:
# Set random seed for reproducibility
torch.manual_seed(42)
np.random.seed(42)

In [4]:
# Configurations
MODEL_NAME = "google-t5/t5-small"
TRANSLATION_PREFIX = "translate from English to French: "
MAX_LENGTH = 128
BATCH_SIZE = 16
NUM_TRAIN_EPOCHS = 3
LEARNING_RATE = 2e-4

In [5]:
# Load dataset
books = load_dataset("opus_books", "en-fr", split = "train[:1000]")
books = books.train_test_split(test_size = 0.2, seed = 42)
books

/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


DatasetDict({
    train: Dataset({
        features: ['id', 'translation'],
        num_rows: 800
    })
    test: Dataset({
        features: ['id', 'translation'],
        num_rows: 200
    })
})

In [6]:
# translation sample example
books["train"]["translation"][0]

{'en': 'The old lady, shaky and worn with age, never ceased chatting and laughing.',
 'fr': 'La vieille dame, cassée, tremblante, ne cessait de causer gaiement et de rire.'}

In [7]:
# Initialize tokenizer
tokenizer = AutoTokenizer.from_pretrained(MODEL_NAME)

In [8]:
def preprocessing_function(examples):

    """
    Preprocess the translation dataset by tokenizing inputs and targets.

    Args:
        examples (dict): Batch of translation examples

    Returns:
        dict: Tokenized model inputs
    """

    inputs = [TRANSLATION_PREFIX + ex["en"] for ex in examples["translation"]]
    targets = [ex["fr"] for ex in examples["translation"]]

    model_inputs = tokenizer(
        inputs,
        text_target = targets,
        max_length = MAX_LENGTH,
        truncation = True
    )

    return model_inputs

In [9]:
# Tokenize dataset
tokenized_books = books.map(
    preprocessing_function,
    batched = True,
    remove_columns = books["train"].column_names
)

Map:   0%|          | 0/800 [00:00<?, ? examples/s]

Map:   0%|          | 0/200 [00:00<?, ? examples/s]

In [10]:
# Data collator
data_collator = DataCollatorForSeq2Seq(
    tokenizer = tokenizer,
    model = MODEL_NAME,
    return_tensors = "pt"
)

In [11]:
# Load evaluation metric
metric = evaluate.load("sacrebleu")

def postprocess_text(preds, labels):

    """
    Postprocess predictions and labels for metric computation.

    Args:
        preds (list): Model predictions
        labels (list): Ground truth labels

    Returns:
        tuple: Processed predictions and labels
    """

    preds = [pred.strip() for pred in preds]
    labels = [[label.strip()] for label in labels]

    return preds, labels

In [12]:
def compute_metrics(eval_pred):
    """
    Compute translation metrics.

    Args:
        eval_pred (EvalPrediction): Model predictions and labels

    Returns:
        dict: Evaluation metrics
    """
    preds, labels = eval_pred

    # Decode predictions and labels
    if isinstance(preds, tuple):
        preds = preds[0]

    # Convert to token IDs
    decoded_preds = tokenizer.batch_decode(
        preds.argmax(-1),
        skip_special_tokens=True
    )

    # Handle labels (replace -100 with pad token)
    decoded_labels = tokenizer.batch_decode(
        np.where(labels != -100, labels, tokenizer.pad_token_id), # where labels equal -100 replace with pad token
        skip_special_tokens=True
    )

    # Postprocess
    decoded_preds, decoded_labels = postprocess_text(
        decoded_preds, decoded_labels
    )

    # Compute BLEU score
    result = metric.compute(
        predictions = decoded_preds,
        references = decoded_labels
    )

    # Add generation length metric
    prediction_lens = [
        np.count_nonzero(pred) for pred in preds.argmax(-1)
    ]

    return {
        "bleu": result["score"],
        "gen_len": np.mean(prediction_lens),
        "bleu_normalized": round(result["score"], 4)
    }

In [13]:
# Quantization configuration
quantization_config = BitsAndBytesConfig(
    load_in_4bit = True ,
    bnb_4bit_compute_dtype = 'bfloat16',
    bnb_4bit_quant_type = 'nf4',
    bnb_4bit_use_double_quant = True ,
)

In [14]:
# Load base model with quantization
base_model = AutoModelForSeq2SeqLM.from_pretrained(
    MODEL_NAME,
    quantization_config = quantization_config,
)

# insted of storing activations in ram to be used in backword pass
# save some and calculate the other during backword pass, this saves some memory
base_model.gradient_checkpointing_enable()

# Prepare model for k-bit training
model = prepare_model_for_kbit_training(base_model)

`low_cpu_mem_usage` was None, now default to True since model is quantized.


In [15]:
# Find target modules for LoRA
target_modules = set()
for name, module in base_model.named_modules():
    if isinstance(module, torch.nn.Linear):
        module_name = name.split('.')[-1]
        target_modules.add(module_name)
target_modules = list(target_modules)
target_modules

['wo', 'q', 'lm_head', 'wi', 'o', 'k', 'v']

In [16]:
# Configure LoRA
lora_config = LoraConfig(
    r = 4,
    lora_alpha = 2,
    lora_dropout = 0.1,
    bias = 'none',
    task_type = TaskType.SEQ_2_SEQ_LM,
    target_modules = target_modules
)

# Get PEFT model
lora_model = get_peft_model(model, lora_config)
lora_model.print_trainable_parameters()

trainable params: 671,232 || all params: 61,177,856 || trainable%: 1.0972


/usr/local/lib/python3.11/dist-packages/peft/tuners/tuners_utils.py:543: UserWarning: Model with `tie_word_embeddings=True` and the tied_target_modules=['lm_head'] are part of the adapter. This can lead to complications, for example when merging the adapter or converting your model to formats other than safetensors. See for example https://github.com/huggingface/peft/issues/2018.
  warnings.warn(


In [17]:
# Training arguments
training_args = TrainingArguments(
    output_dir = "./translation_results",
    overwrite_output_dir = True,
    num_train_epochs = NUM_TRAIN_EPOCHS,
    per_device_train_batch_size = BATCH_SIZE,
    per_device_eval_batch_size = BATCH_SIZE,
    learning_rate = LEARNING_RATE,
    warmup_ratio = 0.1,
    weight_decay = 0.01, # L2 regularization
    logging_dir = './logs',
    logging_strategy = 'steps',
    logging_steps = 50,
    evaluation_strategy = 'epoch',
    save_strategy = 'epoch',
    load_best_model_at_end = True,
    metric_for_best_model = 'bleu',
    dataloader_pin_memory = True, # data intially loadded into CPU then copied to GPU, set to True for transfer directly to GPU, it's faster
    bf16 = True,
    push_to_hub = False,  # Set to True if you want to push to HuggingFace Hub
)

/usr/local/lib/python3.11/dist-packages/transformers/training_args.py:1575: FutureWarning: `evaluation_strategy` is deprecated and will be removed in version 4.46 of 🤗 Transformers. Use `eval_strategy` instead
  warnings.warn(


In [18]:
# Initialize Trainer
trainer = Trainer(
    model = lora_model,
    args = training_args,
    train_dataset = tokenized_books['train'],
    eval_dataset = tokenized_books['test'],
    data_collator = data_collator,
    compute_metrics = compute_metrics,
    processing_class = tokenizer
)

# Train the model
trainer.train()

wandb: WARNING The `run_name` is currently set to the same value as `TrainingArguments.output_dir`. If this was not intended, please specify a different run name by setting the `TrainingArguments.run_name` parameter.
wandb: Currently logged in as: abdulrahman-ahmed20072 (abdulrahman-ahmed20072-none) to https://api.wandb.ai. Use `wandb login --relogin` to force relogin
wandb: Using wandb-core as the SDK backend.  Please refer to https://wandb.me/wandb-core for more information.


/usr/local/lib/python3.11/dist-packages/torch/_dynamo/eval_frame.py:632: UserWarning: torch.utils.checkpoint: the use_reentrant parameter should be passed explicitly. In version 2.5 we will raise an exception if use_reentrant is not passed. use_reentrant=False is recommended, but if you need to preserve the current default behavior, you can pass use_reentrant=True. Refer to docs for more details on the differences between the two variants.
  return fn(*args, **kwargs)
`use_cache=True` is incompatible with gradient checkpointing. Setting `use_cache=False`...


Epoch,Training Loss,Validation Loss,Bleu,Gen Len,Bleu Normalized
1,2.083500,1.903999,4.936789,106.800000,4.936800
2,2.058100,1.882716,4.983657,106.800000,4.983700
3,2.034300,1.873490,5.061911,106.800000,5.061900


Passing a tuple of `past_key_values` is deprecated and will be removed in Transformers v4.48.0. You should pass an instance of `EncoderDecoderCache` instead, e.g. `past_key_values=EncoderDecoderCache.from_legacy_cache(past_key_values)`.
/usr/local/lib/python3.11/dist-packages/peft/utils/save_and_load.py:230: UserWarning: Setting `save_embedding_layers` to `True` as embedding layers found in `target_modules`.
  warnings.warn("Setting `save_embedding_layers` to `True` as embedding layers found in `target_modules`.")
/usr/local/lib/python3.11/dist-packages/torch/_dynamo/eval_frame.py:632: UserWarning: torch.utils.checkpoint: the use_reentrant parameter should be passed explicitly. In version 2.5 we will raise an exception if use_reentrant is not passed. use_reentrant=False is recommended, but if you need to preserve the current default behavior, you can pass use_reentrant=True. Refer to docs for more details on the differences between the two variants.
  return fn(*args, **kwargs)
/usr/lo

TrainOutput(global_step=150, training_loss=2.0586392720540365, metrics={'train_runtime': 87.5308, 'train_samples_per_second': 27.419, 'train_steps_per_second': 1.714, 'total_flos': 50135221075968.0, 'train_loss': 2.0586392720540365, 'epoch': 3.0})

In [19]:
# Inference function
def translate_text(text, max_length=MAX_LENGTH):

    """
    Translate text from English to French.

    Args:
        text (str): Input text to translate
        max_length (int): Maximum length of generated translation

    Returns:
        str: Translated text
    """

    # Prepare input
    inputs = tokenizer(
        TRANSLATION_PREFIX + text,
        return_tensors = "pt",
        max_length = max_length,
        truncation = True
    ).to(lora_model.device)

    # Set model to evaluation mode
    lora_model.eval()

    # Generate translation
    with torch.no_grad():
        outputs = lora_model.generate(
            **inputs,
            max_length = max_length,
            num_return_sequences = 1,
            do_sample = False  # Use greedy decoding
        )

    # Decode and return translation
    return tokenizer.decode(outputs[0], skip_special_tokens=True)

In [20]:
# Example usage
example_text = "Hello, how are you?"

print(f"Original: {example_text}")
print(f"Translation: {translate_text(example_text)}")

Original: Hello, how are you?
Translation: Bonjour, comment êtes-vous?
